Notebook to investigate the QFT library
==============
here we do the reverse engeneering the tobias programm

In [1]:
import sftrident.qft as qft
import numpy as np
import Konstanten as con
import Grid_cms as grid

/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: n

IndentationError: expected an indented block (Grid_cms.py, line 6)

In [ ]:

def eps_laser(l):
    return (l==1)*qft.MinkowskiVector( [0,1,0,0] ) + (l==2)*qft.MinkowskiVector( [0,0,1,0] )

def eps_photon(l):
    return (l==1)*qft.MinkowskiVector([0,np.cos(theta_ph)*np.cos(phi_ph),np.cos(theta_ph)*np.sin(phi_ph),-np.sin(theta_ph)])\
           +(l==2)*MinkowskiVector( [0,-np.sin(phi_ph),np.cos(phi_ph),0])


w_laser     = 0.75/con.m
w_photon    = w_laser
w_cms       = w_laser
w           = w_laser
sqrt_s      = 2.*w_laser

pplusp = w_laser

a0            = 1.0
sigma         = 25 #pulse length
w_laser_l     = 1.e0
w_photon_l    = 4.e0
ksi           = con.pi/4.

Genau_theta   = 10
Genau_gamma   = 5

Harmonic_max  = 2


Envelope      = 'cos^2'

m_eff         = con.m*np.sqrt(1+0.5*a0**2)


theta_ph      = 0.
phi_ph        = 0.
phi_pos       = 0.


Abstand   = 2.0

y0_min  = lambda l: np.log ( w_cms/con.m - np.sqrt( w_cms**2/con.m**2 - 1./l ) )
y1_max  = lambda l: np.log ( w_cms/con.m + np.sqrt( w_cms**2/con.m**2 - 1./l ) )

y_z    = qft.parray(np.linspace(y0_min(Harmonic_max + Abstand),y1_max(Harmonic_max + Abstand),Genau_theta),axis=1)[np.newaxis,:]
#y_z = qft.parray(np.array([y_zOLD[0,1]]))[np.newaxis,:]
#y_z = qft.parray(np.log(pplusp/m_perp))

print "yz: %s"%(y_z)
print "yz.shape: %s"%(str(y_z.shape))

#p_perp=qft.parray(1.0)

#m_perp  = sqrt(1 + p_perp**2)
p_perp  = qft.parray(grid.Grid_rap(y_z[0,:], w_cms, 0 , Harmonic_max + Abstand , Genau_gamma , a0)) #p_T
m_perp  = np.sqrt(1 + p_perp**2)
print "pT: %s"%(p_perp)
print "pT.shape: %s"%(str(p_perp.shape))




In [ ]:
k_laser = qft.MinkowskiVector([w_laser,0,0,-w_laser])
print "k(laser): %s"%np.asarray(k_laser)
n1 = qft.MinkowskiVector([1, np.sin(theta_ph)*np.cos(phi_ph), np.sin(theta_ph)*np.sin(phi_ph), np.cos(theta_ph)])
print "norm vec n (photon): %s"%np.asarray(n1)
k_photon = w_photon*n1
print "k (photon): %s"%np.asarray(k_photon)
p_pos = qft.MinkowskiVector([ m_perp*np.cosh(y_z),
                            p_perp*np.cos(phi_pos),
                            p_perp*np.sin(phi_pos),
                            m_perp*np.sinh(y_z)] )
print "mom Positron: %s"%np.asarray(p_pos)
print "mom0 Positron: %s"%np.asarray(m_perp*np.cosh(y_z))

q_pos = p_pos + (con.m**2*a0**2)/(4.*(k_laser*p_pos))*k_laser # eff Mom Pos


p_el_plus = k_photon.plus_component() - p_pos.plus_component()
p_el_1= k_photon._1() - p_pos._1()
p_el_2 = k_photon._2()                 -     p_pos._2()

p_el_minus = (con.m**2+p_el_1**2+p_el_2**2)/p_el_plus

p_el_0     = 0.5*(p_el_plus + p_el_minus)
p_el_3     = 0.5*(p_el_plus - p_el_minus)
p_el       = qft.MinkowskiVector([ p_el_0, p_el_1 ,p_el_2, p_el_3 ])
#print "mom Electron: %s"%np.asarray(p_el)

eps_m      = eps_laser(1)*np.cos(ksi)-1j*eps_laser(2)*np.sin(ksi)
eps_p      = eps_laser(1)*np.cos(ksi)+1j*eps_laser(2)*np.sin(ksi)